<a href="https://colab.research.google.com/github/marcory-hub/swift-yolo/blob/main/Swift_YOLO_192.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Make Swift-YOLO model 192x192 px


This colab notebook is based on the colab notebooks on [ssmca-model-zoo](https://github.com/Seeed-Studio/sscma-model-zoo)

**Category:** Object Detection

**Algorithm:** [Swift-YOLO](configs/swift_yolo/swift_yolo_tiny_1xb16_300e_coco.py)

**Dataset:** Custom dataset from Roboflow (requires Roboflow account and API key)

---

## Setup Instructions

Follow these steps before running the notebook:

### Step 1: Enable GPU Acceleration

1. Click `Runtime` → `Change runtime type`
2. Select `GPU` in `Hardware accelerator`
3. Click `Save`

**Expected environment** (verify in first code cell output):
- GPU: T4 or higher
- Python 3.11.13

---

### Step 2: Configure Roboflow API Key

⚠️ **Required for dataset download**

1. Go to your [Roboflow Settings](https://app.roboflow.com/settings/api)
2. Copy your private API key
3. In Colab, click the **key icon** (🔑) on the **left sidebar** to open Secrets
4. Click `+ Add Secret`
5. Add a new secret:
   - **Name**: `ROBOFLOW_API_KEY` (exact name, case-sensitive)
   - **Value**: Paste your API key
6. **Enable access**: Toggle the switch next to `ROBOFLOW_API_KEY` to **ON** (this grants the notebook access to the secret)

---

### Step 3: Configure Roboflow Project Information

⚠️ **Required for dataset download**

1. In Roboflow, navigate to your project → `Versions` → `Train` → `Download Dataset`
2. Under `Image and Annotation Format`, select **`COCO`**
3. Click `Show download code` → `Continue`
4. Select **`Jupyter`** as the format
5. **Copy the 3rd line** that looks like:
   ```
   project = rf.workspace("your-workspace").project("your-project")
   ```
6. In the Colab notebook, find the code block marked:
   ```
   ⚠️ PASTE YOUR PROJECT CODE LINE IN THE CODE BLOCK BELOW
   ```
7. **Paste** the copied line into that code block
8. **Adjust the version number** in the line below if needed (default = 1)

---

### Step 4: Adjust Number of Classes

⚠️ **IMPORTANT: Update class count if your dataset has a different number of classes**

- **Default**: 4 classes
- **Action**: Search for all occurrences of `num_classes` or class count settings in the notebook and update them to match your dataset

**Example**: If your dataset has 5 classes, change `num_classes=4` to `num_classes=5` in all relevant code blocks.

---

## Ready to Run

After completing all setup steps above, you can run the notebook cells in order. The notebook will:
1. Download your dataset from Roboflow
2. Train the Swift-YOLO model
3. Export the model as `int8_vela.tflite` for Grove Vision AI V2


# Check PGU and Python version

In [ ]:
# Check if GPU is available
!nvidia-smi

# Check Python version
!python --version

## Prerequisites
### Setup SSCMA
Clone the [repository](https://github.com/Seeed-Studio/ModelAssistant) and install the dependencies.

(you can ignore the ERROR)

In [ ]:
!pip install ethos-u-vela
!git clone https://github.com/Seeed-Studio/ModelAssistant.git -b 2.0.0  #clone the repo
%cd ModelAssistant
!. ./scripts/setup_colab.sh

### Download the pretrain model weights file

In [ ]:
%mkdir -p Swift-YOLO_192
!wget -c https://files.seeedstudio.com/sscma/model_zoo/detection/gesture/swift_yolo_1xb16_300e_coco_sha1_adda465db843aae8384c90c82e223c2cd931cad2.pth -O Swift-YOLO_192/pretrain.pth

### Download the dataset

Download original model from Seeed

In [ ]:
%mkdir -p Swift-YOLO_192
!pip install roboflow
import os
from google.colab import userdata
os.environ["ROBOFLOW_API_KEY"] = userdata.get("ROBOFLOW_API_KEY")
from roboflow import download_dataset
dataset = download_dataset("https://universe.roboflow.com/rsp/paper-aaj0p/dataset/33", "coco")
dataset_path = dataset.location
import shutil
shutil.move(dataset_path, "Swift-YOLO_192/dataset")


# ⚠️ PASTE YOUR PROJECT CODE LINE IN THE CODE BLOCK BELOW

In [ ]:
%mkdir -p Swift-YOLO_192
!pip install roboflow

import os
import shutil
from google.colab import userdata
from roboflow import Roboflow

# Setup API Key from Colab Secrets.
# Make sure you add you Roboflow API key to the secrets (under key symbol, left side bar in colab)
api_key = userdata.get("ROBOFLOW_API_KEY")
rf = Roboflow(api_key=api_key)

# PASTE YOUR PROJECT CODE LINE HERE:
project = rf.workspace("pcv-eagdp").project("vvel")
version = project.version(1)

dataset = version.download("coco")
dataset_path = dataset.location
dest_path = "Swift-YOLO_192/dataset"
if os.path.exists(dest_path):
    shutil.rmtree(dest_path) # Remove existing to prevent nested folders

shutil.move(dataset_path, dest_path)

print(f"Dataset successfully moved to: {dest_path}")

## Train a model with SSCMA
All the training parameters are in the `config.py` file, you can change the parameters to train your own model.

Below are explanations of some common parameters. You can also refer to the [documentation](https://sensecraftma.seeed.cc/tutorials/config) for more details.
- `data_root` - the datasets path.
- `epochs`- the train epochs. **we use 10 epochs as an example**.
- `batch_size` - the batch size.
- `height` - the image height.
- `width` - the image width.
- `load_from` - the pretrained model path.
- `num_classes` - the number of classes.

You can overwrite the parameters in the `config.py` file by using the `--cfg-options` argument.

In [ ]:
!sscma.train configs/swift_yolo/swift_yolo_tiny_1xb16_300e_coco.py \
--cfg-options  \
    work_dir=Swift-YOLO_192 \
    num_classes=4 \
    epochs=10  \
    height=192 \
    width=192 \
    data_root=Swift-YOLO_192/dataset/ \
    load_from=Swift-YOLO_192/pretrain.pth

## Export the model
After training, you can export the model to the format for deployment. SSCMA supports exporting to ONNX, and TensorFlow Lite at present.
You can also refer to the [documentation](https://sensecraftma.seeed.cc/tutorials/export/overview) for more details.

```bash
python3 tools/export.py \
    "<CONFIG_FILE_PATH>" \
    "<CHECKPOINT_FILE_PATH>"
```

In [ ]:
import os
with open('Swift-YOLO_192/last_checkpoint', 'r') as f:
	os.environ['CHECKPOINT_FILE_PATH'] = f.read()

In [ ]:
!sscma.export configs/swift_yolo/swift_yolo_tiny_1xb16_300e_coco.py $CHECKPOINT_FILE_PATH --cfg-options  \
    work_dir=Swift-YOLO_192 \
    num_classes=4 \
    epochs=10  \
    height=192 \
    width=192 \
    data_root=Swift-YOLO_192/dataset/ \
    load_from=Swift-YOLO_192/pretrain.pth

### Evaluate the model
After exporting the model, you can evaluate the model on the test dataset.
You can also refer to the [documentation](https://sensecraftma.seeed.cc/tutorials/export/overview) for more details.


```bash
python3 tools/inference.py \
    "<CONFIG_FILE_PATH>" \
    "<CHECKPOINT_FILE_PATH>"
```

### Evaluate the PyTorch model

In [ ]:
!sscma.inference configs/swift_yolo/swift_yolo_tiny_1xb16_300e_coco.py ${CHECKPOINT_FILE_PATH%.*}.pth \
--cfg-options  \
    work_dir=Swift-YOLO_192 \
    num_classes=4 \
    epochs=10  \
    height=192 \
    width=192 \
    data_root=Swift-YOLO_192/dataset/ \
    load_from=Swift-YOLO_192/pretrain.pth

### Evaluate the ONNX model

In [ ]:
!sscma.inference configs/swift_yolo/swift_yolo_tiny_1xb16_300e_coco.py ${CHECKPOINT_FILE_PATH%.*}_float32.onnx \
--cfg-options  \
    work_dir=Swift-YOLO_192 \
    num_classes=7 \
    epochs=10  \
    height=192 \
    width=192 \
    data_root=Swift-YOLO_192/dataset/ \
    load_from=Swift-YOLO_192/pretrain.pth

### Evaluate the TFLite FLOAT32 model

In [ ]:
!sscma.inference configs/swift_yolo/swift_yolo_tiny_1xb16_300e_coco.py ${CHECKPOINT_FILE_PATH%.*}_float32.tflite \
--cfg-options  \
    work_dir=Swift-YOLO_192 \
    num_classes=7 \
    epochs=10  \
    height=192 \
    width=192 \
    data_root=Swift-YOLO_192/dataset/ \
    load_from=Swift-YOLO_192/pretrain.pth

### Evaluate the TFLite INT8 model

In [ ]:
!sscma.inference configs/swift_yolo/swift_yolo_tiny_1xb16_300e_coco.py ${CHECKPOINT_FILE_PATH%.*}_int8.tflite \
--cfg-options  \
    work_dir=Swift-YOLO_192 \
    num_classes=4 \
    epochs=10  \
    height=192 \
    width=192 \
    data_root=Swift-YOLO_192/dataset/ \
    load_from=Swift-YOLO_192/pretrain.pth

## Deploy the model
After model training, evaluation and export, you can deploy the model to your device.

In [ ]:
%ls -lh Swift-YOLO_192/

# Zip and download the Swift-YOLO_192 model

In [ ]:
import os
import zipfile
from google.colab import files

# Define the paths
folder_path = '/content/ModelAssistant/Swift-YOLO_192'
zip_filename = 'Swift-YOLO_Models.zip'

# Create the zip file
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    # List all items in the directory
    for item in os.listdir(folder_path):
        item_path = os.path.join(folder_path, item)

        # Only add to zip if it's a file (skips all subdirectories)
        if os.path.isfile(item_path):
            zipf.write(item_path, arcname=item)
            print(f"Added to zip: {item}")

# Automatically trigger the download to your computer
files.download(zip_filename)

# Deploy model on [SenseCraft AI platform](https://sensecraft.seeed.cc/ai/model)

1. Click `My Own Models`
2. Select `+ Add Model`
3. Follow the instructions
4. Download the int8_vela.tflite model